In [1]:
import pandas as pd 

In [2]:
df=pd.read_csv("C:\\Users\\DELL\\Downloads\\100_Unique_QA_Dataset.csv")

In [3]:
df.shape

(90, 2)

In [4]:
df

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100
...,...,...
85,Who directed the movie 'Titanic'?,JamesCameron
86,Which superhero is also known as the Dark Knight?,Batman
87,What is the capital of Brazil?,Brasilia
88,Which fruit is known as the king of fruits?,Mango


In [5]:
def tokenize(text):
    text=text.lower()
    text=text.replace("?","")
    text=text.replace(".","")
    return text.split()

In [6]:
df["question"][0]

'What is the capital of France?'

In [7]:
tokenize("What is the capital of France?")

['what', 'is', 'the', 'capital', 'of', 'france']

## tokenization is done now lets make our vocab 


In [8]:
vocab={"unk":0}
def build_vocab(row):
    print(row["question"])
    print(row["answer"])
    tokens_questions=tokenize(row["question"])
    tokens_answer=tokenize(row["answer"])
    merged_texts=tokens_questions+tokens_answer

    for text in (merged_texts):
        if text not in vocab:
            vocab[text]=len(vocab)
        
        

In [9]:
vocab={"unk":0}

In [10]:
def build_vocab(texts):
  #print(texts["question"],texts["answer"])
  tokens_question=tokenize(texts["question"])
  tokens_answer=tokenize(texts["answer"])
  merged_texts=tokens_question+tokens_answer
  for text in merged_texts:
    if text not in vocab:
      vocab[text]=len(vocab)
  return vocab
  


In [11]:
df.apply(build_vocab,axis=1)

0     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
1     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
2     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
3     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
4     {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
                            ...                        
85    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
86    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
87    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
88    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
89    {'unk': 0, 'what': 1, 'is': 2, 'the': 3, 'capi...
Length: 90, dtype: object

In [12]:
vocab

{'unk': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 "'to": 12,
 'kill': 13,
 'a': 14,
 "mockingbird'": 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 "'1984'": 67,
 'george-orwell': 68,
 'currency': 69,
 'un

In [13]:
len(vocab)

326

In [14]:
## no of unique wirds is 91

In [15]:
def indices(texts,vocab):
    indices_list=[]
    tokens=tokenize(texts)
    for tok in tokens:
        if tok in vocab:
            indices_list.append(vocab[tok])
        else:
            indices_list.append(vocab["unk"])
    return indices_list
            

In [16]:
df["answer"][0]

'Paris'

In [17]:
indices("Paris",vocab)

[7]

In [18]:
## now we can convert all questions and answer to indices using dataset and dataloader 

In [19]:
from torch.utils.data import Dataset,DataLoader

In [20]:
import torch

In [21]:
class qadataset(Dataset):
    def __init__(self,df,vocab):
        self.df=df
        self.vocab=vocab
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        
        row=self.df.iloc[idx]
        question_indices=indices(row["question"],self.vocab)
        answer_indices=indices(row["answer"],self.vocab)
        return torch.tensor(question_indices),torch.tensor(answer_indices)
        

In [22]:
qdataset=qadataset(df,vocab)

In [23]:
qdataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [24]:
dataloader=DataLoader(qdataset,batch_size=1,shuffle=True)

In [25]:
len(dataloader)

90

In [26]:
for q,a in dataloader:
  print(q,a)

tensor([[  1,   2,   3, 213,   5,  14, 214, 215]]) tensor([[216]])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([[54]])
tensor([[ 1,  2,  3,  4,  5, 99]]) tensor([[100]])
tensor([[ 78,  79, 290,  81,  19,  14, 291]]) tensor([[85]])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([[128]])
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]]) tensor([[61]])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([[36]])
tensor([[ 42, 301, 302, 118,  14, 303, 304, 159, 305, 306, 307, 308]]) tensor([[309]])
tensor([[ 10, 140,   3, 141, 272,  93, 273,   5,   3, 274]]) tensor([[275]])
tensor([[ 42, 137,   2, 227, 143,   3, 228, 229]]) tensor([[156]])
tensor([[ 10,  96,   3, 104, 241]]) tensor([[242]])
tensor([[  1,   2,   3, 122, 123,  19,   3,  45]]) tensor([[124]])
tensor([[ 42, 137,   2,  62,  39,   3, 324, 325]]) tensor([[6]])
tensor([[  1,   2,   3, 141, 117,  83,   3, 279, 280]]) tensor([[121]])
tensor([[ 42, 137,   2, 138,  39, 176, 271]]) tensor([[99]])
tensor([[ 10,  75, 209]]) tensor([[210]]

In [27]:
import torch.nn as nn

In [58]:
## architecture of model 
class simplernn(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn=nn.RNN(50,64,batch_first=True)
    self.linear=nn.Linear(64,vocab_size)
  def forward(self,question):
    embedding_ques=self.embedding(question)
    hidden,final=self.rnn(embedding_ques)
    output=self.linear(final.squeeze(0))
    return output


In [29]:
## here we understand why not we use sequential container 
##we give one dataset in this architectutre to understandthis 

In [30]:
qdataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [31]:
x=nn.Embedding(324,embedding_dim=50)

In [32]:
x(qdataset[0][0])## first index of first data question wala sirf 

tensor([[ 1.1653,  0.9338, -0.3408, -1.0944, -0.0053, -0.1364, -0.3133,  1.7368,
         -0.5589,  0.5846,  1.3847, -0.7712,  1.6593,  0.9638,  1.0226,  2.0094,
          1.0793, -0.8593, -0.7199,  0.7488,  0.7125, -0.9444, -0.1080, -0.9420,
          0.8200, -0.2924, -0.4153, -0.0826, -1.5420,  0.9003,  2.0360,  0.2431,
          0.8477,  1.3171,  0.1878,  0.9317, -0.2304,  1.2411,  0.8493, -0.4404,
          0.0934,  1.1713,  0.7869,  0.6169,  0.5186,  0.1789, -2.2504, -0.2091,
         -1.3145,  0.4195],
        [ 0.6551,  0.8259,  0.0960,  0.1559, -0.6543, -0.1732,  0.4643, -0.4064,
         -1.6438,  1.4281,  0.2506,  0.7538,  0.4566,  0.5094,  1.9924,  0.4668,
          0.7565,  0.5426, -0.5883,  0.4614,  0.1923,  1.8219, -0.5715, -0.2798,
         -1.1796,  0.4419, -0.4859, -1.5364,  0.7450, -0.1225, -0.4360,  0.0656,
         -0.4888,  0.2272,  1.8769, -0.2569,  0.5505, -0.6211,  0.2718, -0.4077,
          0.3019,  0.6769,  1.8682, -2.5333, -0.8885, -0.8263, -1.6492, -0.1413,


## output main hume 50 dimension ka vector mila
## shape of this tensor 

In [33]:
qdataset[0][0].shape

torch.Size([6])

In [34]:
a=x(qdataset[0][0])

In [35]:
 ## 6 vectorhain sabka size 50 dimension hain 

In [36]:
## lets take 10 question 

In [37]:
x(qdataset[10][0])

tensor([[ 1.1653,  0.9338, -0.3408, -1.0944, -0.0053, -0.1364, -0.3133,  1.7368,
         -0.5589,  0.5846,  1.3847, -0.7712,  1.6593,  0.9638,  1.0226,  2.0094,
          1.0793, -0.8593, -0.7199,  0.7488,  0.7125, -0.9444, -0.1080, -0.9420,
          0.8200, -0.2924, -0.4153, -0.0826, -1.5420,  0.9003,  2.0360,  0.2431,
          0.8477,  1.3171,  0.1878,  0.9317, -0.2304,  1.2411,  0.8493, -0.4404,
          0.0934,  1.1713,  0.7869,  0.6169,  0.5186,  0.1789, -2.2504, -0.2091,
         -1.3145,  0.4195],
        [ 0.6551,  0.8259,  0.0960,  0.1559, -0.6543, -0.1732,  0.4643, -0.4064,
         -1.6438,  1.4281,  0.2506,  0.7538,  0.4566,  0.5094,  1.9924,  0.4668,
          0.7565,  0.5426, -0.5883,  0.4614,  0.1923,  1.8219, -0.5715, -0.2798,
         -1.1796,  0.4419, -0.4859, -1.5364,  0.7450, -0.1225, -0.4360,  0.0656,
         -0.4888,  0.2272,  1.8769, -0.2569,  0.5505, -0.6211,  0.2718, -0.4077,
          0.3019,  0.6769,  1.8682, -2.5333, -0.8885, -0.8263, -1.6492, -0.1413,


In [38]:
qdataset[15][0]

tensor([ 1,  2,  3, 69,  5,  3, 70, 71])

In [39]:
(qdataset[15][0]).shape

torch.Size([8])

In [40]:
aa=x(qdataset[15][0]).shape

## now make second layer 

In [41]:
y=nn.RNN(50,64)

In [42]:
y(a)

(tensor([[-0.5210,  0.2562, -0.0287, -0.2533,  0.2539,  0.3408, -0.0734,  0.9216,
           0.0579,  0.6097, -0.5934,  0.4859, -0.2570,  0.6977, -0.1105,  0.3387,
          -0.0020,  0.4620, -0.1251, -0.4422,  0.3883, -0.6005, -0.0127,  0.2060,
           0.5702,  0.3117,  0.3945, -0.4749, -0.1528, -0.0148,  0.1204,  0.3902,
          -0.3369, -0.0033, -0.6219, -0.4209, -0.1536, -0.8758, -0.5593, -0.6205,
          -0.2039,  0.2884,  0.5648,  0.0414, -0.3951, -0.4130,  0.4691,  0.1356,
           0.6994,  0.3703,  0.4515,  0.4575, -0.3261, -0.4050, -0.4484,  0.3798,
           0.1742, -0.5406,  0.3925, -0.0881,  0.1708,  0.5193,  0.6599, -0.5787],
         [ 0.0848,  0.2307, -0.0743,  0.5946,  0.3483,  0.6746,  0.4046,  0.0954,
           0.5334,  0.3245, -0.1597, -0.6663,  0.6027,  0.3279, -0.2063,  0.2270,
          -0.1043,  0.5678, -0.3578, -0.0115, -0.2222, -0.1012, -0.6506,  0.1392,
          -0.5554, -0.5205,  0.5978,  0.5564,  0.1439,  0.4672,  0.4439, -0.6192,
          -0.78

## yaha par do vector milte hain 

In [43]:
## rnn hiddens state 
y(a)[0].shape

torch.Size([6, 64])

6 vector 64 dimension

In [44]:
b=y(a)[1]

1 vector 64 dimenson

## thats why we don t use sequential container nn .rnn se do output nikal rahe hain 

In [45]:
z=nn.Linear(64,324)

In [46]:
z(b).shape

torch.Size([1, 324])

In [67]:
epochs=20
learning_rate=0.01

In [60]:
criterion=nn.CrossEntropyLoss()

In [61]:
model=simplernn(len(vocab))

In [68]:
optmizers=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [70]:
for epoch in range(epochs):
  total_loss=0
  for question,answer in dataloader:
     optmizers.zero_grad()
     output=model(question)
     #print(output.shape)
     loss=criterion(output,answer[0])
   
     loss.backward()
     optmizers.step()
     total_loss=total_loss+loss.item()
  print((f"epoch {epoch+1},loss:{total_loss}"))

epoch 1,loss:542.1407904624939
epoch 2,loss:337.20170187950134
epoch 3,loss:147.5760926529765
epoch 4,loss:64.52038191957399
epoch 5,loss:35.905603979481384
epoch 6,loss:29.571665367111564
epoch 7,loss:19.774402503157035
epoch 8,loss:25.126072589540854
epoch 9,loss:16.396091716364026
epoch 10,loss:12.981935262447223
epoch 11,loss:10.938104829750955
epoch 12,loss:8.44750883220695
epoch 13,loss:6.176567688584328
epoch 14,loss:5.130922587355599
epoch 15,loss:6.73987127142027
epoch 16,loss:10.022082969779149
epoch 17,loss:5.8015821690205485
epoch 18,loss:8.657224042108282
epoch 19,loss:5.137202845653519
epoch 20,loss:11.50523822545074


In [ ]:
##humara jo answer hain dataloader me 2d vector hain  use 1 d vector hona chahiye 

In [54]:
df["answer"].shape

(90,)

In [71]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [76]:
predict(model, "are you know what is important ")

I don't know
christophercolumbus
